# Social Network Analysis Project: Covid-19 diffusion network in Gibraltar
## Members:
- Joonas Mäenpää (Joonas.Maenpaa@student.oulu.fi) 

- Joakim Junnila (jjunnila18@student.oulu.fi) 

- Mahdi Akbari (mahdi.akbari@oulu.fi) 

## Task 1:
Use a starting date where you consider it to stand for initial state. In the statistics of the country at the chosen, calculate the initial infection as the total number of infection minus the total recovery. Use the official corona statistical source to draw a plot showing the temporal variations of the number of infections and that of the number of recovery. 

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

gib = pd.read_csv('Gibraltar_covid19.csv')

#display(gib)

plt.plot(gib['active_cases'])
plt.grid()
plt.title('Historical data')
plt.ylabel('Number of cases')

plt.plot(gib['cumulative_cases'])
plt.plot(gib['cumulative_removed'])

plt.legend(["Active Cases", "Cumulative Infected", "Cumulative Removed"])

plt.savefig('T1_active_cases.jpeg', dpi = 300)

## Task 2:
We want to carry on the simulation using the SIR epidemic model. Use the implementation provided in NDLIB library to perform the calculus. Set the number of nodes of the network equal to the total population and a very small probability for Erdos random graph of 0.001. Choose a infection probability beta and recovery probability gamma of your choice (you may inspire from the data trend). Run the NDLIB and plot the temporal variation of the Number of infection and recovery over time.

In [ ]:
population = 33706

g = nx.erdos_renyi_graph(n = population, p = 0.001, seed=10) 

In [ ]:
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
def sir_infected(beta_input, gamma_input, g):
          
    model = ep.SIRModel(g, seed=10)

    cfg = mc.Configuration()

    cfg.add_model_parameter('beta', beta_input)
    cfg.add_model_parameter('gamma', gamma_input)
    cfg.add_model_parameter('percentage_infected', 0)

    model.set_initial_status(cfg)

    iteration = model.iteration_bunch(200)

    trend = model.build_trends(iteration)

    viz = DiffusionTrend(model, trend)
    
    infected_model = viz.trends[0]['trends']['node_count'][1]
    
    removed_model = viz.trends[0]['trends']['node_count'][2]
    
    sus_mod = viz.trends[0]['trends']['node_count'][0]
    
    return {'infected_model' : pd.DataFrame(infected_model).astype('float')/population,
            'removed_model' : pd.DataFrame(removed_model).astype('float')/population,
            'sus_mod' : pd.DataFrame(sus_mod).astype('float')/population}

In [ ]:
beta = 0.02

gamma = 0.05

model_us = sir_infected(beta, gamma, g)

infected_model = model_us['infected_model']

removed_model = model_us['removed_model']

sus_mod = model_us['sus_mod']

plt.plot(infected_model)
plt.grid()
plt.title('\u03B2 =' + str(beta) + 'and \u03B3 = ' + str(gamma))
plt.ylabel('Number of cases')

plt.plot(removed_model)
plt.plot(sus_mod)

plt.legend(["Infected Model", "Removed Model", "Susceptible Model"])

plt.savefig('T2_us_modeled.jpeg', dpi = 300)

## Task 3:
Now we want to use the data of official statistics to tune the probability of infection and recovery to find a way to match the variations plotted in 2) with that of 1). Suggest an empirical approach where, for instance, you vary incrementally the values of betta and gamma until you visualize a figure infections and recovery count closely match that of official statistics. 

In [ ]:
import seaborn as sns

betta = np.arange(0.01, 0.02, 0.001)
gama = np.arange(0.0, 0.5, 0.05)

dif_sqr = np.zeros((10,10))

for i in range(10): 
    
    for j in range(10): 
        
        model_us = sir_infected(betta[i], gama[j], g)

        infected_model = model_us['infected_model']

        removed_model = model_us['removed_model']

        dif_sqr[i, j] = abs(max(np.array(infected_model).ravel())*population - max(gib['active_cases'][0:200]))

In [ ]:
df = pd.read_csv('dif_sqr_cat.csv', index_col = 0)

df

ax = sns.heatmap(df, annot=True, linewidths= 0.5, cbar=False)

plt.xlabel('\u03B3')
plt.ylabel('\u03B2 * 100')

plt.savefig('T3_heatmat_maxdif.jpeg', dpi = 300, figsize=(30, 10))

In [ ]:
betta_ave = 0.015

gamma_ave = 0.1

model_us = sir_infected(betta_ave, gamma_ave, g)

infected_model = model_us['infected_model']

removed_model = model_us['removed_model']

plt.plot(infected_model*population)
plt.grid()
plt.ylabel('Number of cases')
plt.title('\u03B2 =' + str(betta_ave) + 'and \u03B3 = ' + str(gamma_ave))

plt.plot(gib['active_cases'], )
plt.legend(["Modeled Infection", "Observed Infection"])

plt.savefig('T3_2_AveBG.jpeg', dpi = 300)

## Task 4:
Now we want to use the official dataset statistics to estimate the probability of infection and recovery. Suggest a simple approach to calculate these attributes using the available historical dataset. Then input these values to the SIR model and run the simulation to display the variation of the infections and recovery. Discuss the relevance of the SIR model for this purpose. 

In [ ]:
gamma_ave = 0.1

betta_ave = 0.0741

model_us = sir_infected(betta_ave, gamma_ave, g)

infected_model = model_us['infected_model']

removed_model = model_us['removed_model']

plt.plot(infected_model*population)
plt.grid()
plt.ylabel('Number of cases')
plt.title('\u03B2 = ' + str(betta_ave) + ' and \u03B3 = ' + str(gamma_ave))

plt.plot(gib['active_cases'])
plt.legend(["Modeled Infection", "Observed Infection"])

plt.savefig('T4_AveBG.jpeg', dpi = 300)

## Task 5:
Now we want to treat the death count provided in the statistics. Consider using the SI model for this purpose. Similarly to 1), draw the timely evolution of the number of death. 

In [ ]:
plt.plot(gib['cumulative_deaths'][0:200] + gib['cumulative_cases'][0:200])
plt.grid()
plt.title('Historical data')
plt.ylabel('Cumulative removed cases')

plt.savefig('T5_death_cases.jpeg', dpi = 300)

## Task 6:
Next use the implementation provided in NDLIB for the SI model and suggest a simple model to generate the simulated model that displays the total number of infected (here we assume any one who is infected or died is in I (i.e. infected) set).  

In [ ]:
def si_infected(beta_input, g):
          
    model = ep.SIModel(g, seed=10)

    cfg = mc.Configuration()

    cfg.add_model_parameter('beta', beta_input)
    cfg.add_model_parameter('percentage_infected', 0)

    model.set_initial_status(cfg)

    iteration = model.iteration_bunch(200)

    trend = model.build_trends(iteration)

    viz = DiffusionTrend(model, trend)
    
    infected_model = viz.trends[0]['trends']['node_count'][1]
       
    sus_mod = viz.trends[0]['trends']['node_count'][0]
    
    return {'infected_model' : pd.DataFrame(infected_model).astype('float')/population,
            'sus_mod' : pd.DataFrame(sus_mod).astype('float')/population}

In [ ]:
betta_input = 0.007

model_us = si_infected(betta_input, g)

infected_model = model_us['infected_model']

sus_model = model_us['sus_mod']

plt.plot(infected_model)
plt.grid()
plt.ylabel('Number of cases')
plt.title('\u03B2 = ' + str(betta_input))

plt.plot(sus_model)

plt.legend(['Infected Model', 'Susceptible Model'])


plt.savefig('T6_us_SImodeled.jpeg', dpi = 300)

## Task 7:
Suggest an empirical and incremental variation of the infection probability in SI model until the death variation is close to the real dataset. Discuss the relevance of such approach and probability value.

In [ ]:
bettas = np.arange(0.004, 0.009, 0.001) 

dif = []

for betta_input in bettas:
    
    model_us = si_infected(betta_input, g)

    model = model_us['infected_model']*population 

    obs = gib['cumulative_deaths'][0:200] + gib['cumulative_cases'][0:200]

    a = sum(((np.array(model) - np.array(obs))**2).ravel())/10**12 

    dif.append(a)

In [ ]:
plt.plot(bettas*100, dif)
plt.grid()
plt.ylabel('Square difference')
plt.xlabel('\u03B2 * 100')

plt.savefig('T7_1_sqDif.jpeg', dpi = 300)

In [ ]:
betta_input = 0.0055

model_us = si_infected(betta_input, g)

infected_model = model_us['infected_model']

plt.plot(infected_model*population)
plt.grid()
plt.ylabel('Number of cases')
plt.title('\u03B2 = ' + str(betta_input))

plt.plot(gib['cumulative_deaths'] + gib['cumulative_cases'])
plt.legend(["Modeled Infection", "Observed Infection"])

plt.savefig('T7_2_SI.jpeg', dpi = 300)

## Task 8:
Consider the SEIRS (Susceptible-Exposed-Infected-Recovered-Susceptible) model described in link below. Set an initial value of parameters of the model and display the temporal evolution of the infection and recovery counts:
- https://github.com/ryansmcgee/seirsplus. 

Read below to understand parameters of SEIRSNetworkModel:
- https://github.com/ryansmcgee/seirsplus/wiki/SEIRSNetworkModel-class

In [ ]:
from seirsplus.models import *
from seirsplus.networks import *
from seirsplus.sim_loops import *
from seirsplus.utilities import *

baseGraph = g

G_normal = custom_exponential_graph(baseGraph, scale=100)

# Social distancing interactions:
G_distancing = custom_exponential_graph(baseGraph, scale=10)

# Quarantine interactions:
G_quarantine = custom_exponential_graph(baseGraph, scale=5)

model = SEIRSNetworkModel(G=G_normal, beta=0.155, sigma=1/5.2, gamma=1/12.39, mu_I=0.0004, p=0.5,
                          G_Q=G_quarantine, beta_Q=0.155, sigma_Q=1/5.2, gamma_Q=1/12.39, mu_0=0.0004,
                          theta_E=0.02, theta_I=0.02, phi_E=0.2, phi_I=0.2, psi_E=1.0, psi_I=1.0, q=0.5,
                          initI=10)

model.run(T=200)

model.figure_infections(plot_S = False,
                        plot_E = 'line',
                        plot_I = 'line',
                        plot_R = 'line',
                        figsize = (15, 10))

## Task 9:
Similarly to 3), suggest an empirical and possible an incremental approach to attempt to match the infection and recovery counts with that of real dataset.

In [ ]:
beta = 0.125

baseGraph = g

G_normal = custom_exponential_graph(baseGraph, scale=100)

# Social distancing interactions:
G_distancing = custom_exponential_graph(baseGraph, scale=10)

# Quarantine interactions:
G_quarantine = custom_exponential_graph(baseGraph, scale=5)

model = SEIRSNetworkModel(G=G_normal, beta=beta, sigma=1/5.2, gamma=1/12.39, mu_I=0.0004, p=0.5,
                          G_Q=G_quarantine, beta_Q=0.155, sigma_Q=1/5.2, gamma_Q=1/12.39, mu_0=0.0004,
                          theta_E=0.02, theta_I=0.02, phi_E=0.2, phi_I=0.2, psi_E=1.0, psi_I=1.0, q=0.5,
                          initI=10)

model.run(T=200)

S = model.numS
E = model.numE
I = model.numI

t = model.tseries

plt.plot(t, I)
plt.grid()
plt.ylabel('Number of cases')

plt.plot(gib['active_cases'])
plt.legend(["SEIR Infection", "Observed Infection"])
plt.title('\u03B2 = ' + str(beta))

plt.savefig('T9_b'+ str(beta) +'.jpeg', dpi = 300)

## Task 10:
Similarly to 4), use the official statistics to infer the infection, recovery probabilities and other parameters of the model. Then run the model again and display the new graph showing the variations of the infections and recovery counts.

In [ ]:
baseGraph = g

G_normal = custom_exponential_graph(baseGraph, scale=100)

# Social distancing interactions:
G_distancing = custom_exponential_graph(baseGraph, scale=10)

# Quarantine interactions:
G_quarantine = custom_exponential_graph(baseGraph, scale=5)

model = SEIRSNetworkModel(G=G_normal, beta=0.155, sigma=1/5.2, gamma=1/12.39, mu_I=0.0004, p=0.5,
                          G_Q=G_quarantine, beta_Q=0.155, sigma_Q=1/5.2, gamma_Q=1/12.39, mu_0=0.0004,
                          theta_E=0.02, theta_I=0.02, phi_E=0.2, phi_I=0.2, psi_E=1.0, psi_I=1.0, q=0.5,
                          initI=10)

model.run(T=200)

model.figure_infections(plot_S = False,
                        plot_E = 'line',
                        plot_I = 'line',
                        plot_R = 'line',
                        figsize = (15, 10))